# Bouncing ball

## Basic (C style) implementation

In [1]:
import ipywidgets as widgets
from ipycanvas import RoughCanvas, hold_canvas
from threading import Thread, Event
from time import sleep

BALL_SIZE=10
BALL_SPEED=2
SCREEN_WIDTH=400
SCREEN_HEIGHT=200

canvas = RoughCanvas(width=SCREEN_WIDTH, height=SCREEN_HEIGHT)


def draw_ball(x, y, size):
    with hold_canvas():
        canvas.clear()
        canvas.stroke_circle(x, y, size)
        canvas.fill_circle(x, y, size)


draw_ball(50, 50, BALL_SIZE)


def thread_controller(thread_func):
    thread = None
    stop_event = Event()
    start_btn = widgets.Button(icon="play", tooltip="start")
    stop_btn = widgets.Button(icon="stop", tooltip="stop")

    def start_thread(*args):
        global thread
        print("Starting thread")
        thread = Thread(target=thread_func, kwargs={"should_stop": stop_event.is_set})
        thread.start()
        start_btn.disabled = True
        stop_btn.disabled = False

    def stop_thread(*args):
        global thread
        stop_event.set()
        thread.join()
        print("Thread stopped")
        thread = None
        stop_event.clear()
        start_btn.disabled = False
        stop_btn.disabled = True

    start_btn.on_click(start_thread)
    stop_btn.on_click(stop_thread)
    stop_btn.disabled = True

    return widgets.HBox([start_btn, stop_btn])


def game_func(should_stop):
    x, y = 50, 50
    x_speed, y_speed = BALL_SPEED, BALL_SPEED
    print("game starting")
    while not should_stop():
        x += x_speed
        y += y_speed
        if x >= SCREEN_WIDTH - BALL_SIZE or x <= BALL_SIZE:
            x_speed = -x_speed
        if y >= SCREEN_HEIGHT - BALL_SIZE or y <= BALL_SIZE:
            y_speed = -y_speed
        draw_ball(x, y, BALL_SIZE)
        sleep(0.01)
    print("game stopped")


widgets.VBox([thread_controller(game_func), canvas])
